In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import requests


In [ ]:
# Load data
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')

# Merge datasets
movies = movies.merge(credits, on='title')

# Select relevant columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# Define conversion functions
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter += 1
    return L

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ", ""))
    return L1

# Data preprocessing
movies.dropna(inplace=True)
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert).apply(lambda x: x[0:3])
movies['crew'] = movies['crew'].apply(fetch_director)
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

# Combine tags
movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

# Drop old columns and prepare tags for vectorization
new = movies.drop(columns=['overview', 'genres', 'keywords', 'cast', 'crew'])
new['tags'] = new['tags'].apply(lambda x: " ".join(x))


In [ ]:
# Vectorization
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()

# Compute similarity
similarity = cosine_similarity(vector)

# Save model and similarity matrix
pickle.dump(new, open('/content/movie_list.pkl', 'wb'))
pickle.dump(similarity, open('/content/similarity.pkl', 'wb'))


In [ ]:
def fetch_poster(movie_id):
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US"
    data = requests.get(url).json()
    poster_path = data['poster_path']
    full_path = f"https://image.tmdb.org/t/p/w500/{poster_path}"
    return full_path

def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommended_movie_names = []
    recommended_movie_posters = []
    for i in distances[1:11]:
        movie_id = new.iloc[i[0]].movie_id
        recommended_movie_posters.append(fetch_poster(movie_id))
        recommended_movie_names.append(new.iloc[i[0]].title)
    return recommended_movie_names, recommended_movie_posters


In [ ]:
# Test recommendation
movie = 'Avatar'
recommended_movies = recommend(movie)
print("Recommended Movies:", recommended_movies)


Recommended Movies: (['Titan A.E.', 'Small Soldiers', "Ender's Game", 'Aliens vs Predator: Requiem', 'Independence Day', 'Krull', 'Lifeforce', 'Battle: Los Angeles', 'Falcon Rising', 'Edge of Tomorrow'], ['https://image.tmdb.org/t/p/w500//el2iHk3LTJWfEnwrvcRkvWY501G.jpg', 'https://image.tmdb.org/t/p/w500//2nuUjSzHsoYlRvTPmLo7m7gCQry.jpg', 'https://image.tmdb.org/t/p/w500//vrEpLNkv30qw7JiVyorgR6NOWDm.jpg', 'https://image.tmdb.org/t/p/w500//jCyJN1vj8jqJJ0vNw4hDH2KlySO.jpg', 'https://image.tmdb.org/t/p/w500//p0BPQGSPoSa8Ml0DAf2mB2kCU0R.jpg', 'https://image.tmdb.org/t/p/w500//2dyMDv6W3ugMfaC8DEXPiqMcurD.jpg', 'https://image.tmdb.org/t/p/w500//953hMDf9G2ZRIEs97M6iFIYWtWF.jpg', 'https://image.tmdb.org/t/p/w500//jloyGeVYZSxM9zsLFvVOWuj2ey4.jpg', 'https://image.tmdb.org/t/p/w500//4wnfTO8mvqcTU62YMkUeKq49VMT.jpg', 'https://image.tmdb.org/t/p/w500//xjw5trHV7Mwo61P0kCTy8paEkgO.jpg'])
